In [2]:
import tensorflow as tf

sequence_length = 7
vocab_size = 128
embedding_size = 5

# emb captures embeddings for the entire vocabulary
# Usually obtained through word2vec training using some corpus (example: news data)
# But for this toy example, we are randomly generating them - [128 x 5] matrix
# We are also assuming we have only 128 words in this vocab set
emb = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name="emb")

# Place holder to hold batch of sentences
# In this example, sentence is limited to max 7 words
input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")

# Look up word embeddings from emb for each sentence (pay attention to matrix shape)
emb_input = tf.nn.embedding_lookup(emb, input_x)

# Add one more dimension at end - channel, so that we can use conv2d later
# conv2d operator requires input to be in [batch, height, width, channel]
# in our example we have only channel (in case of images, we may have 3 channels)
# Convert data from [batch, height, width] => [batch, height, width, channel]
# Remember - we are just adding one dimension to matrix, it can only have one channel
emb_input_expanded = tf.expand_dims(emb_input, -1)

# Create 2 filters of each with heights [2, 3, 4]
# Filter with height 2 will cover 2 consecutive words each time
# width of filter is the embedding dimension size 
filter_sizes = [2, 3, 4] 
num_filters = 2

pooled_outputs = []
for i, filter_size in enumerate(filter_sizes):
    with tf.name_scope("conv-maxpool-%s" % filter_size):
        filter_shape = [filter_size, embedding_size, 1, num_filters]

        # Initialize each filter's weights
        # It is 3-dimesional filter, but depth is only 1 channel
        W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")

        # Apply Filter on input [batch, seq_length, embedding_size, 1]
        # Move one word at time. In images
        conv = tf.nn.conv2d(emb_input_expanded, 
            W, strides=[1, 1, 1, 1], 
            padding="VALID", name="conv")

        # Apply relu - max(0, output) as activation functiona 
        h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")

        # Apply max pooling on result activation map
        pooled = tf.nn.max_pool( h, ksize=[1, sequence_length - filter_size + 1, 1, 1], 
            strides=[1, 1, 1, 1], padding='VALID', name="pool")
        pooled_outputs.append(pooled)

# Create session and initialize weight matrices
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Write graph definition to a file, so that tensorboard can read it ...
writer = tf.summary.FileWriter("./cnn_text", graph=tf.get_default_graph())

# Input setup, each row is one sentence, each column represents one word
# Pick each words embeddings from previously trained embeddings
# Batch contains 4 sentences, each sentence is one training sample (add labels later)
batch_x = [
    [1, 4, 6, 8, 20, 2, 8], 
    [11, 14, 16, 18, 20, 12, 18],
    [21, 24, 26, 28, 20, 22, 28],
    [31, 34, 36, 38, 20, 32, 38],
]

sess.run(emb_input_expanded, feed_dict={input_x:batch_x})


array([[[[ 0.50518584],
         [ 0.73519874],
         [ 0.55573463],
         [ 0.41568804],
         [-0.28822255]],

        [[-0.46627617],
         [ 0.82505345],
         [ 0.41570282],
         [ 0.3787756 ],
         [-0.94981098]],

        [[-0.86273956],
         [-0.6960845 ],
         [ 0.86087441],
         [ 0.31347966],
         [-0.70282483]],

        [[ 0.54513073],
         [ 0.99571466],
         [ 0.08626652],
         [ 0.6693058 ],
         [-0.70080662]],

        [[-0.30533576],
         [ 0.59425211],
         [-0.73013449],
         [ 0.56492805],
         [-0.0988152 ]],

        [[ 0.95599389],
         [-0.89279747],
         [-0.44778395],
         [-0.29341841],
         [-0.48117065]],

        [[ 0.54513073],
         [ 0.99571466],
         [ 0.08626652],
         [ 0.6693058 ],
         [-0.70080662]]],


       [[[-0.61933088],
         [ 0.41693616],
         [-0.42060399],
         [ 0.62799668],
         [ 0.62960315]],

        [[-0.32618475]

In [3]:
# of shape [4] => just one dimension (simple array of 4 elements)
a = tf.Variable(tf.random_normal([4]))
sess.run(a.initializer)
sess.run(a)

array([-0.71179408,  1.50984275, -0.14741264, -0.77934337], dtype=float32)

In [4]:
# Let's expand it by adding one dimension at end (indicated by -1)
# Converts into [4, 1]   => 4 rows, each row with one element
sess.run(tf.expand_dims(a, -1))

array([[-0.71179408],
       [ 1.50984275],
       [-0.14741264],
       [-0.77934337]], dtype=float32)

In [5]:
sess.run(tf.expand_dims(a, -1)).shape

(4, 1)

In [8]:
e1 = tf.Variable([1,2,3,4])
sess.run(e1.initializer)

In [15]:
a = sess.run(e1)
print(a.shape)
b = sess.run(tf.expand_dims(e1, -1))
print(b.shape)
b

(4,)
(4, 1)


array([[1],
       [2],
       [3],
       [4]], dtype=int32)